<a href="https://colab.research.google.com/github/Cinnes8850/Energy_Infrastructure/blob/main/single_node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install
!pip install -q pyomo
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 71.8 MB/s eta 0:00:00


In [ ]:
# load libraries and solver
from pyomo.environ import *
from pyomo.opt import SolverFactory, TerminationCondition
import sys
sys.meta_path = [hook for hook in sys.meta_path if hasattr(hook, "find_spec")]
import gurobipy as gp  # import the installed package

In [ ]:
# create PYOMO optimization model
mdl = ConcreteModel()

In [ ]:
# Sets
mdl.Consumers = Set(initialize=["k1", "k2"])
mdl.Generators = Set(initialize=["g1", "g2"])

In [ ]:
# We can check the whole model with:
mdl.pprint()

2 Set Declarations
    Consumers : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'k1', 'k2'}
    Generators : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'g1', 'g2'}

2 Declarations: Consumers Generators


In [ ]:
# Bid prices for demand and supply as Python dictionaries
# (we can use also Param() to create a parameter with Pyomo)
price_dem = {"k1": 50, "k2":  0}
price_gen = {"g1": 20, "g2": 60}
# print(price_dem)

In [ ]:
# Max quantity for demand and supply as Python dictionaries
max_dem = {"k1": 100, "k2": 100}
max_gen = {"g1": 30, "g2": 100}

In [ ]:
# Variables for demand and generation
mdl.dem = Var(mdl.Consumers, domain=NonNegativeReals)
mdl.gen = Var(mdl.Generators, domain=NonNegativeReals)

In [ ]:
# we can check individual components
mdl.dem.pprint()
mdl.gen.pprint()

dem : Size=2, Index=Consumers
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     k1 :     0 :  None :  None : False :  True : NonNegativeReals
     k2 :     0 :  None :  None : False :  True : NonNegativeReals
gen : Size=2, Index=Generators
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     g1 :     0 :  None :  None : False :  True : NonNegativeReals
     g2 :     0 :  None :  None : False :  True : NonNegativeReals


In [ ]:
# Constraints
def dem_max_rule(self, k):
    return mdl.dem[k] <= max_dem[k]
mdl.dem_max_constrait = Constraint(mdl.Consumers, rule=dem_max_rule)

In [ ]:
# check
mdl.dem_max_constrait.pprint()

dem_max_constrait : Size=2, Index=Consumers, Active=True
    Key : Lower : Body    : Upper : Active
     k1 :  -Inf : dem[k1] : 100.0 :   True
     k2 :  -Inf : dem[k2] : 100.0 :   True


In [ ]:
def gen_max_rule(self, g):
    return mdl.gen[g] <= max_gen[g]
mdl.gen_max_constrait = Constraint(mdl.Generators, rule=gen_max_rule)

In [ ]:
# check
mdl.gen_max_constrait.pprint()

gen_max_constrait : Size=2, Index=Generators, Active=True
    Key : Lower : Body    : Upper : Active
     g1 :  -Inf : gen[g1] :  30.0 :   True
     g2 :  -Inf : gen[g2] : 100.0 :   True


In [ ]:
# power balance
tot_dem = sum(mdl.dem[k] for k in mdl.Consumers)
tot_gen = sum(mdl.gen[g] for g in mdl.Generators)
mdl.power_balance = Constraint(expr=tot_dem == tot_gen)

# check
mdl.power_balance.pprint()

power_balance : Size=1, Index=None, Active=True
    Key  : Lower : Body                                    : Upper : Active
    None :   0.0 : dem[k1] + dem[k2] - (gen[g1] + gen[g2]) :   0.0 :   True


In [ ]:
# Objective function
total_benefit_consumers = sum(price_dem[k] * mdl.dem[k] for k in mdl.Consumers)
total_cost_generators = sum(price_gen[k] * mdl.gen[k] for k in mdl.Generators)

mdl.obj = Objective(expr=total_benefit_consumers - total_cost_generators, sense=maximize)

# check
mdl.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 50*dem[k1] + 0*dem[k2] - (20*gen[g1] + 60*gen[g2])


In [ ]:
# check the whole model
mdl.pprint()

2 Set Declarations
    Consumers : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'k1', 'k2'}
    Generators : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'g1', 'g2'}

2 Var Declarations
    dem : Size=2, Index=Consumers
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         k1 :     0 :  None :  None : False :  True : NonNegativeReals
         k2 :     0 :  None :  None : False :  True : NonNegativeReals
    gen : Size=2, Index=Generators
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         g1 :     0 :  None :  None : False :  True : NonNegativeReals
         g2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 50*dem[k1] + 0*dem[k2] - (20*g

In [ ]:
# We have to tell Pyomo that we want dual variables
# In Pyomo, the existence of an active Suffix with the name "dual"
# that has an import style suffix direction will cause constraint
# dual information to be collected into the solver results
# (assuming the solver supplies dual information).
mdl.dual = Suffix(direction=Suffix.IMPORT)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
# Create an object representing the solver, e.g. cplex, gurobi, or mosek
solver = SolverFactory("gurobi")

In [ ]:
# Cplex options in interactive format
# solver.options["timelimit"] = 1800    # time limit in seconds

# solver.options["simplex tolerances optimality"] = 1e-9  # optimality tolerance
# solver.options["simplex tolerances feasibility"] = 1e-9 # feasibility tolerance

# useful for MILP problems, i.e. involving binary variables
# solver.options["mip tolerances integrality"] = 0        # integrality tolerance
# solver.options["mip tolerances mipgap"] = 0             # mixed integer optimality gap tolerance
# solver.options["mip tolerances absmipgap"] = 0          # absolute mixed integer optimality gap tolerance

# solve the optimization problem
results = solver.solve(mdl, tee=True)

Read LP format model from file /tmp/tmpd36emqry.pyomo.lp
Reading time = 0.00 seconds
x1: 5 rows, 4 columns, 8 nonzeros
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
QCPDual  1

Optimize a model with 5 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x54350e52
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   0.000000e+00   2.000000e-06      0s
Extra simplex iterations after uncrush: 1
       1    9.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iter

In [ ]:
# other useful arguments
# solve(mdl, tee=True, keepfiles=True, symbolic_solver_labels=True)
# solve(mdl, tee=True, keepfiles=False, warmstart=True)
# solve(mdl, tee=True, keepfiles=True, solnfile=file.sol)
# solve(mdl, tee=True, keepfiles=False, warmstart=True, warmstart_file=file.sol)

# ALWAYS check solver's termination condition
if results.solver.termination_condition != TerminationCondition.optimal:
    raise Exception
else:
    print(results.solver.status)
    print(results.solver.termination_condition)
    print(results.solver.termination_message)
    # print(results.solver.time)

ok
optimal
Model was solved to optimality (subject to tolerances), and an optimal solution is available.


In [ ]:
# print solved variables
mdl.dem.pprint()
mdl.gen.pprint()

# access single values
print("demand k1=",mdl.dem["k1"].value)

# print objective function value
print("welfare=",value(mdl.obj))

# print the market price,
# i.e. the dual variable of
# the power balance constraint
print("market price=%.2f" % (mdl.dual[mdl.power_balance]))

dem : Size=2, Index=Consumers
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     k1 :     0 :  30.0 :  None : False : False : NonNegativeReals
     k2 :     0 :   0.0 :  None : False : False : NonNegativeReals
gen : Size=2, Index=Generators
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     g1 :     0 :  30.0 :  None : False : False : NonNegativeReals
     g2 :     0 :   0.0 :  None : False : False : NonNegativeReals
demand k1= 30.0
welfare= 900.0
market price=50.00
